### Tải các thư viện cần thiết

In [4]:
!pip install -qq datasets evaluate transformers[sentencepiece]
!pip install -qq accelerate
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [5]:
import os
import math
import torch
import pandas as pd

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from huggingface_hub import notebook_login
from datasets import Dataset

### Load dữ liệu

In [6]:
df = pd.read_csv('/content/poem_dataset2.csv', index_col=0)
df

,title,content,source,url
0,“Bạn xấu như chiếc bóng”,Bạn xấu như chiếc bóng\nCứ bám riết theo anh\n...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,“Chơi thân không có nghĩa”,Chơi thân không có nghĩa\nKhông cãi nhau bao g...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,“Có thể buồn chút ít”,"Có thể buồn chút ít\nMột mình, không người yêu...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...
100,Ám ảnh,Tôi giật mình đánh rơi\nCon mèo con xuống giến...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/T%C3%B4-H%C3%A0/%C3%81...
101,Ám ảnh sông xưa,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
102,Áng dương không biết sầu,Áng dương không biết sầu\nNằm mãi ở trên cao\n...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
103,Anh,Cây bút gẫy trong tay\nCặn mực khô đáy lọ\nÁnh...,19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...


In [7]:
df['content'][0]

'Bạn xấu như chiếc bóng\nCứ bám riết theo anh\nKhi anh sáng, rực rỡ\nNhư mặt trời long lanh\n\nNhưng họ sẽ biến mất\nKhi trời phủ mây đen\nTức là khi anh đói\nTrong túi không có tiền'

### Chuẩn bị dữ liệu
Một bài thơ gồm nhiều khổ thơ, một khổ thơ có 4 dòng thơ, mỗi dòng thơ có 5 chữ. Để giảm phức tạp, ta sẽ dùng mỗi một khổ thơ là 1 data sample

In [8]:
def split_content(content):
    samples = []
    poem_parts = content.split('\n\n')

    for poem_part in poem_parts:
        poem_in_lines = poem_part.split('\n')
        if len(poem_in_lines) == 4:
            samples.append(poem_in_lines)
    return samples

df['content'] = df['content'].apply(lambda x: split_content(x))

In [9]:
df.head()

,title,content,source,url
0,“Bạn xấu như chiếc bóng”,"[[Bạn xấu như chiếc bóng, Cứ bám riết theo anh...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,“Cái làm ta hạnh phúc”,"[[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhi...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,“Chiều vừa xốp trên tay”,"[[Chiều vừa xốp trên tay, Chợt nghe thoáng ong...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,“Chơi thân không có nghĩa”,"[[Chơi thân không có nghĩa, Không cãi nhau bao...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,“Có thể buồn chút ít”,"[[Có thể buồn chút ít, Một mình, không người y...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...


In [10]:
# Hàm df.explode() sẽ giúp ta tách các phần tử trong một list thành các hàng mới
df_exploded = df.explode('content')
df_exploded.reset_index(drop=True, inplace=True)
df_exploded = df_exploded.dropna(subset=['content'])
df_exploded.head()

,title,content,source,url
0,“Bạn xấu như chiếc bóng”,"[Bạn xấu như chiếc bóng, Cứ bám riết theo anh,...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,“Bạn xấu như chiếc bóng”,"[Nhưng họ sẽ biến mất, Khi trời phủ mây đen, T...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,“Cái làm ta hạnh phúc”,"[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhiề...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
3,“Cái làm ta hạnh phúc”,"[Rồi thêm chút công việc, Cho ta làm hàng ngày...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,“Chiều vừa xốp trên tay”,"[Chiều vừa xốp trên tay, Chợt nghe thoáng ong ...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...


In [11]:
# chuyển sang dạng string đối với từng content
df_exploded['content'] = df_exploded['content'].apply(lambda x: '\n'.join(x))
df_exploded.head()

,title,content,source,url
0,“Bạn xấu như chiếc bóng”,Bạn xấu như chiếc bóng\nCứ bám riết theo anh\n...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,“Bạn xấu như chiếc bóng”,Nhưng họ sẽ biến mất\nKhi trời phủ mây đen\nTứ...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
3,“Cái làm ta hạnh phúc”,Rồi thêm chút công việc\nCho ta làm hàng ngày\...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...


In [12]:
TEST_SIZE = 0.1
poem_dataset = Dataset.from_pandas(df_exploded)
poem_dataset = poem_dataset.train_test_split(test_size=TEST_SIZE)

In [13]:
poem_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'content', 'source', 'url', '__index_level_0__'],
        num_rows: 182
    })
    test: Dataset({
        features: ['title', 'content', 'source', 'url', '__index_level_0__'],
        num_rows: 21
    })
})

### Tiền xử lý dữ liệu  

In [14]:
MODEL_NAME = 'danghuy1999/gpt2-viwiki'

tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/773k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/431k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

In [15]:
tokenizer.pad_token = tokenizer.eos_token
MAX_SEQ_LEN = 100

def preprocess_function(row):
    return tokenizer(
        row['content'],
        max_length=MAX_SEQ_LEN,
        padding='max_length',
        truncation=True
    )

tokenized_poem_dataset = poem_dataset.map(
    preprocess_function,
    batched=True,
    # num_proc=4,
    remove_columns=poem_dataset['train'].column_names
)

Map:   0%|          | 0/182 [00:00<?, ? examples/s]

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

In [16]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

### Huấn luyện mô hình

In [17]:
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [18]:
training_args = TrainingArguments(
    output_dir="gpt2_viet_poem_generation",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    fp16=True
)

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_poem_dataset["train"],
    eval_dataset=tokenized_poem_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

<ipython-input-19-511dd46a6e8e>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: noinaycoanh094 (noinaycoanh094-hanoi-university-of-civil-engineering) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=230, training_loss=5.959920070482337, metrics={'train_runtime': 1387.0952, 'train_samples_per_second': 1.312, 'train_steps_per_second': 0.166, 'total_flos': 92881152000000.0, 'train_loss': 5.959920070482337, 'epoch': 10.0})

### Inference

In [20]:
prompt = "Học học nữa học mãi\n"
device = "cuda" if torch.cuda.is_available() else "cpu"
inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
outputs = model.generate(
    inputs,
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    repetition_penalty=1.2
)
results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
results = results[0]
print()
for line in results.split("\n"):
    print(line)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Học học nữa học mãi
Cả hai bên nhau
Không người muốn vào tâm trí
nguyệt vàng hơn trên
Hai đồng không khí có thể
Sê "c Speak một cách"
Đôi khi có ý nghĩ và nói
Người ta gặp bạn
Có thể thấy
